# Дистрибутивный анализ *то есть*

In [1]:
import openpyxl
import re
import pymystem3
from nltk.tokenize import word_tokenize
from pymystem3 import Mystem
import gensim
import logging
import nltk.data
import pandas as pd
import urllib.request
from bs4 import BeautifulSoup
from nltk.corpus import stopwords
from gensim.models import word2vec
import warnings
warnings.filterwarnings('ignore')
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
%matplotlib inline
from nltk.corpus import stopwords

## Работа с таблицей по векам

In [2]:
wb = openpyxl.load_workbook('/Users/annaaksenova/Desktop/Работа к Кувшинской/То есть выборка по векам.xlsx')
sheet = wb['21'] # Имя листа

In [3]:
m = Mystem()

In [4]:
v1 = []
# Собираем примеры из таблицы
for v in sheet.iter_rows(min_row=2, min_col=21, max_col=21, max_row=1001, values_only=True):
    if v[0] == None:
        v1.append('')
    else:
        v1.append(v[0].replace(u'\xa0', ' '))
v2 = []
for v in sheet.iter_rows(min_row=2, min_col=22, max_col=22, max_row=1001, values_only=True):
    if v[0] == None:
        v2.append('')
    else:
        item = v[0].replace(u'\xa0', ' ')
        item = re.sub('\[.+?\]', '', item)
        item = re.sub('[Тт]о есть', 'тоесть', item) # \b убирает случаи что есть
        v2.append(item)
values = [v1[i]+v2[i] for i in range(1000)]
print(values[:10])

[' Я засунула пластинку и блюдо в один пакет, положила под сиденье и ― надо знать наши самолёты советских времён, лишённые кондиционеров, ― в конце пути в Ереван достала из-под сиденья пластинку, принявшую форму таза, тоесть блюда.   ', ' Само название места происходит от того, что здесь врачевали «косимых» ― тоесть увечных.   ', ' Я даже не могу найти концов, тоесть понять, кто же мне, собственно, должен платить.   ', 'Никаких провокаций ― известные имена-брэнды.  Расчёт на то, что, даже если зритель не шибкоприобщён к балету и не знает, кто такая Аньес Летестю, он непременно отреагирует на Opera National de Paris.  Или хотя бы на «Лебединое озеро». Или хотя бы на Paris.  Плюс гарантия качества: звёзды подобраны заслуженные, но при этом, что важно, действующие: спешно доставленные напароходе прямо из Парижа, поднимешь крышку ― пар! тоесть готовые доказывать свой статус каждым спектаклем.   ', '  «Чтобы не быть выдавленными с рынка, ― говорит академик Алексей Макаров, ― необходимы жест

In [81]:
tagged_corp = []
for sentence in values:
    ana = m.analyze(sentence)
    tagged_corp.append(ana)

In [82]:
print(len(tagged_corp))

1000


In [83]:
final = []
count = 1
for sent in tagged_corp:
    sentence = []
    for word in sent:
        if 'analysis' in word and word['analysis'] != []:
            gr = word['analysis'][0]['gr']
            pos = gr.split('=')[0].split(',')[0]
            lex = word['analysis'][0]['lex']
            token = word['text']
            if pos == 'V' or pos == 'S' or pos == 'A' or pos == 'ADV':
                sentence.append((token, lex, pos))
    if sentence == []:
        print(sent)
        print(count)
    else:
        final.append(sentence)
    count+=1

In [85]:
with open('sentences.txt', 'w') as fw:
    for sentence in final:
        for word in sentence:
            for part in word:
                fw.write(part + ' ')
        fw.write('\n')

In [ ]:
sentence_counts = []
for sentence in final:
    left_sum = 0
    right_sum = 0
    count_l = 0
    count_r = 0
    ind = 0 # индикатор было ли вхождение то есть
    for word in sentence:
        if word[0] == 'тоесть':
            ind = 1
        if not ind:
            count_l += 1
            left_sum += ?? # тут вектор слова
        elif ind:
            count_r += 1
            left_sum += ?? # тут вектор слова
        
    count = (left_sum, right_sum)
    sentence_counts.append(count)

In [5]:
import re
part_sent = []

# тут я немного криво сохранила текстовый файл с сервера, просто считываю данные номер предложения + косинус
# при этом я выкинула примеры, где вектор хотябы одного контекста nan
with open('part_sim.txt') as fh: 
    lines = fh.readlines()
for line in lines:
    line = line.strip()
    a = re.search(r'([0-9]+)(-?0\.[0-9]+)', line)
    number = a.group(1)
    cos = a.group(2)
    part_sent.append([number, cos])


In [8]:
wb = openpyxl.load_workbook('/Users/annaaksenova/Desktop/Работа к Кувшинской/То есть выборка по векам.xlsx')
sheet = wb['21']
for i in part_sent: # для каждой пары номер-косинус вытаскиваем из таблицы разметку
    i.append(sheet['Q{}'.format(str(int(i[0])+1))].value)

In [10]:
from pprint import pprint
pprint(part_sent)

[['292', '0.89379615', 'Иными словами'],
 ['953', '0.8557891', 'следствие'],
 ['215', '0.8420359', 'следствие'],
 ['301', '0.81316423', 'пояснение'],
 ['557', '0.8128834', 'следствие'],
 ['505', '0.81279266', 'импликатура'],
 ['598', '0.80335826', 'пояснение'],
 ['62', '0.79922545', 'Обобщение'],
 ['213', '0.7939003', 'переспрос'],
 ['826', '0.78499955', 'пояснение'],
 ['189', '0.7849252', 'импликатура'],
 ['104', '0.7845668', 'пояснение'],
 ['729', '0.77797437', 'пояснение'],
 ['896', '0.7732643', 'следствие'],
 ['814', '0.7731954', 'отмена сказанного'],
 ['725', '0.7725307', 'пояснение'],
 ['8', '0.7691949', 'Импликатура'],
 ['202', '0.766552', 'импликатура'],
 ['517', '0.7628418', 'следствие'],
 ['230', '0.755937', 'следствие'],
 ['722', '0.75262', 'следствие'],
 ['314', '0.7489022', 'пояснение'],
 ['530', '0.7418438', 'пояснение'],
 ['278', '0.7413057', 'пояснение'],
 ['698', '0.73294836', 'пояснение'],
 ['309', '0.73140705', 'импликатура'],
 ['471', '0.7305995', 'переспрос'],
 ['4